### Author: Reinaldo Maciel
### Date: June 9th 2020

## Objective:

Given a Bank customer, build a neural network based classifier that can determine whether they will leave
or not in the next 6 months. 

In [1]:
### Importing libraries
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

/Users/reinaldodeoliveiramaciel/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/reinaldodeoliveiramaciel/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/reinaldodeoliveiramaciel/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/reinaldodeoliveiramaciel/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/reinaldodeoliveir

In [ ]:
### reading data
